In [ ]:
import os
import numpy as np
import trimesh

def world_to_voxel(coords, origin, spacing):
    """World 좌표를 voxel 인덱스로 변환"""
    return ((coords - origin) / spacing).round().astype(int)

def label_faces_from_mask(mesh_folder, mask_folder, output_folder, spacing=(1.0, 1.0, 1.0)):
    os.makedirs(output_folder, exist_ok=True)

    mesh_files = [f for f in os.listdir(mesh_folder) if f.endswith('.obj')]

    for mesh_file in mesh_files:
        base_name = os.path.splitext(mesh_file)[0]
        mesh_path = os.path.join(mesh_folder, mesh_file)
        mask_path = os.path.join(mask_folder, base_name + ".npy")

        if not os.path.exists(mask_path):
            print(f"❌ 마스크 없음: {mask_path}")
            continue

        # Load mesh and mask
        mesh = trimesh.load(mesh_path)
        mask = np.load(mask_path)
        mask_shape = mask.shape
        spacing = np.array(spacing)

        # 중심 기반 origin 추정
        mask_center = np.array(mask_shape) / 2
        mesh_center = mesh.centroid
        origin = mesh_center - mask_center * spacing

        # face 중심 → voxel 좌표
        face_centers = mesh.vertices[mesh.faces].mean(axis=1)
        face_voxels = world_to_voxel(face_centers, origin, spacing)

        # 유효 영역 마스크
        D, H, W = mask_shape
        valid_mask = (
            (face_voxels[:, 0] >= 0) & (face_voxels[:, 0] < D) &
            (face_voxels[:, 1] >= 0) & (face_voxels[:, 1] < H) &
            (face_voxels[:, 2] >= 0) & (face_voxels[:, 2] < W)
        )

        # face 레이블 생성
        face_labels = np.zeros(len(mesh.faces), dtype=np.int64)
        valid_coords = face_voxels[valid_mask]
        mask_values = mask[valid_coords[:, 0], valid_coords[:, 1], valid_coords[:, 2]]
        face_labels[valid_mask] = mask_values.astype(np.int64)

        # 저장
        label_path = os.path.join(output_folder, base_name + "_label.npy")
        np.save(label_path, face_labels)
        print(f"✅ 저장됨: {label_path} (결절 face: {face_labels.sum()})")


In [ ]:
label_faces_from_mask(
    mesh_folder="/path/to/mesh",        # .obj 파일들이 있는 폴더
    mask_folder="/path/to/masks",       # .npy 마스크가 있는 폴더
    output_folder="/path/to/output",    # 결과 라벨 저장 위치
    spacing=(1.0, 1.0, 1.0)             # voxel 해상도
)